In [20]:
# @title Libraries
import numpy as np
import pandas as pd
import scipy
from scipy import signal
import matplotlib.pyplot as plt
import os
from matplotlib.mlab import psd

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
import tensorflow as tf
import xgboost as xgb
import os
import tabulate
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report
from IPython.display import clear_output
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import numpy as np
from scipy.io import loadmat
import os
from pywt import wavedec
from functools import reduce
from scipy import signal
from scipy.stats import entropy
from scipy.fft import fft, ifft
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tensorflow import keras as K
import matplotlib.pyplot as plt
import scipy
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,cross_validate
from tensorflow.keras.layers import Dense, Activation, Flatten, concatenate, Input, Dropout, LSTM, Bidirectional,BatchNormalization,PReLU,ReLU,Reshape
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, Model, load_model
import matplotlib.pyplot as plt;
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from tensorflow import keras
from tensorflow.keras.layers import Conv1D,Conv2D,Add
from tensorflow.keras.layers import MaxPool1D, MaxPooling2D
import seaborn as sns
import random

#@title Run this to import classifiers
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


In [19]:
!rm -r /content/Data
!git clone https://github.com/Neurotech-X-Columbia/Data.git
!rm -r /content/Data/SSVEP/TF

Cloning into 'Data'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 230 (delta 23), reused 209 (delta 18), pack-reused 7
Receiving objects: 100% (230/230), 101.35 MiB | 20.23 MiB/s, done.
Resolving deltas: 100% (23/23), done.
Checking out files: 100% (150/150), done.


In [21]:
num_chans = 16
fs = 125
notch_f = 60
Q_factor = 30
lp_f = 2
iir_hp_2 = signal.iirfilter(2, 2. / fs, btype='highpass')
a, b = signal.iirnotch(notch_f, Q_factor, fs)

def filtering(d):
    filtered_sig = signal.filtfilt(iir_hp_2[0], iir_hp_2[1], d, padlen=0)
    filtnnotch_sig = signal.filtfilt(a, b, filtered_sig, padlen=0)
    return filtnnotch_sig

In [22]:
datadir = r'/content/Data/SSVEP'
category_folders = [x[0] for x in os.walk(datadir) if 'Bottom' in x[0] or 'Top' in x[0]]
category_folders

['/content/Data/SSVEP/GG/BottomMiddle',
 '/content/Data/SSVEP/GG/TopRight',
 '/content/Data/SSVEP/GG/TopLeft',
 '/content/Data/SSVEP/GG/BottomRight',
 '/content/Data/SSVEP/GG/BottomLeft',
 '/content/Data/SSVEP/JP/BottomMiddle',
 '/content/Data/SSVEP/JP/TopRight',
 '/content/Data/SSVEP/JP/TopLeft',
 '/content/Data/SSVEP/JP/BottomRight',
 '/content/Data/SSVEP/JP/BottomLeft',
 '/content/Data/SSVEP/NG/BottomMiddle',
 '/content/Data/SSVEP/NG/TopRight',
 '/content/Data/SSVEP/NG/TopLeft',
 '/content/Data/SSVEP/NG/BottomRight',
 '/content/Data/SSVEP/NG/BottomLeft',
 '/content/Data/SSVEP/MC/BottomMiddle',
 '/content/Data/SSVEP/MC/TopRight',
 '/content/Data/SSVEP/MC/TopLeft',
 '/content/Data/SSVEP/MC/BottomRight',
 '/content/Data/SSVEP/MC/BottomLeft']

In [23]:
ids = ['TopLeft', 'TopRight', 'BottomLeft', 'BottomMiddle', 'BottomRight']
positions = [[],[],[],[],[]] #each subarray corresponds to position above
for i, pos in enumerate(ids):
    for f in category_folders:
        if f.endswith(pos):
            for csv in os.listdir(f):
                positions[i].append(np.genfromtxt(os.path.join(f, csv), delimiter=',')[1:])

In [ ]:
plt.psd(filtering(positions[0][0][15,150:]), Fs=fs)

In [24]:
featnlab = []
for ii in range(5):
    for bruh, tri in enumerate(positions[ii]):
        for chan in [14, 15]:
            if not (np.isnan(tri).any()):
                pow_fxx, fxx = psd(filtering(tri[chan, 125:]), Fs=fs) # what is the 125 here
                featnlab.append(np.append(pow_fxx, ii))

In [25]:
fnl = np.array(featnlab)
print(fnl.shape)
np.random.shuffle(fnl)

(200, 130)


In [27]:
x = fnl[:, :129]
y = fnl[:, 129:]
y_cold = y 
#y_one_hot = pd.DataFrame(y). --> uncomment these lines to one-hot encode and run a deep-learning model
#y_one_hot = pd.get_dummies(y_one_hot[0])
#y = y_one_hot.to_numpy()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2)

In [ ]:
# Use this cell to conduct hyper-paramter optimization for various classifiers
results = []
depth = range(15, 20)
estimators = range(15, 20)
max_f = range(15, 20)
c = [0.001, 0.01, 0.1, 1, 10, 100]
for i in depth:
  for j in estimators:
    for k in max_f:
      model = RandomForestClassifier(max_depth=i, n_estimators=j, max_features=k).fit(x_train, y_train)
      preds = model.predict(x_test)
      acc = accuracy_score(y_test, preds)
      results.append([i, j, k, acc])
results = pd.DataFrame(results)
results = results.sort_values(by=[3])
results



In [32]:
# Ensemble model here
from collections import Counter
m1 = KNeighborsClassifier(3).fit(x_train, y_train) # optimized
m2 = SVC(kernel= "linear", C=0.07).fit(x_train, y_train) # optimized
m3 = xgb.XGBClassifier(3).fit(x_train, y_train) # optimized
m4 = RandomForestClassifier(max_depth=16, n_estimators=16, max_features=17).fit(x_train, y_train) # optimized

def most_frequent(List):
    counter = 0
    num = List[0]
    for i in List:
        curr_frequency = List.count(i)
        if(curr_frequency> counter):
            counter = curr_frequency
            num = i
    if counter == 0:
      return List[2]
    return num

#neurotech website, cincinnati childrens reasearch, summer program 

preds = []
for i in range(len(x_test)):
  preds1 = m1.predict(np.expand_dims(x_test[i], axis = 0))
  preds2 = m2.predict(np.expand_dims(x_test[i], axis = 0))
  preds3 = m3.predict(np.expand_dims(x_test[i], axis = 0))
  preds4 = m4.predict(np.expand_dims(x_test[i], axis = 0))
  predictions = [preds1, preds2, preds3, preds4]
  preds.append(most_frequent(predictions))

preds = np.array(preds)
acc = accuracy_score(y_test, preds)
acc


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for examp

0.65

In [ ]:
classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.03),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

results = []

for i, classifier in enumerate(classifiers):
  classifier.fit(x_train, y_train)
  preds = classifier.predict(x_test)
  acc = accuracy_score(y_test, preds)
  results.append( [f'{classifier}', acc] )

results = pd.DataFrame(results)
results



In [ ]:
inputs = tf.keras.Input(shape=(x_train.shape[1],))
z = tf.keras.layers.Dense(256, activation='relu')(inputs)
z = tf.keras.layers.Dropout(0.05)(z)
outputs = tf.keras.layers.Dense(5, activation='softmax')(z)

model_cnn = tf.keras.Model(inputs, outputs)

In [ ]:
model_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

batch_size = 128
epochs = 1000

history = model_cnn.fit(
    x_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks = [
    tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=500,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
    )]
)